In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [6]:
image = cv2.imread(r"C:\Users\richa\Downloads\img.png")
keyimage = cv2.imread(r"C:\Users\richa\Downloads\imgmask.png", cv2.IMREAD_GRAYSCALE)
binary_mask = (keyimage // 255).astype(np.uint8)

cv2.imshow("image", image)
cv2.imshow("keyimage", keyimage)
cv2.waitKey(0)

-1

In [7]:
features = []

hist_b = cv2.calcHist([image], [0], binary_mask, [256], [0, 256])
hist_g = cv2.calcHist([image], [1], binary_mask, [256], [0, 256])
hist_r = cv2.calcHist([image], [2], binary_mask, [256], [0, 256])

hist_b = hist_b.flatten()
hist_g = hist_g.flatten()
hist_r = hist_r.flatten()
hists = [hist_b, hist_g, hist_r]

col_b = hist_b.argmax() / hist_b.shape[0]
col_g = hist_g.argmax() / hist_g.shape[0]
col_r = hist_r.argmax() / hist_r.shape[0]

features += [col_b, col_g, col_r]


In [8]:

contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    real_area = cv2.contourArea(largest_contour)
    
    radius = cv2.minEnclosingCircle(largest_contour)
    radius = int(radius)
    circle_area = np.pi * radius ** 2

    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    rect_area = cv2.contourArea(box)
    
    
    circle_area_ratio = real_area / circle_area
    rect_area_ratio = real_area / rect_area
    
    total_area_ratio = (image.shape[0] * image.shape[1]) / real_area
    
    features += [circle_area_ratio, rect_area_ratio, total_area_ratio]
    
else:
    raise ValueError("No contours found in binary mask")



ValueError: No contours found in binary mask